In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
data_train = np.load('data/pixelized_sample.npz')
data_labels = np.load('data/pixelized_sample_class.npz')

In [3]:
x_train = data_train['arr_0']
x_labels = data_labels['arr_0']

In [4]:
category_mapping = {90: 'SN1a', 67: 'SN1a-91bg', 52: 'SN1ax', 42: 'SN2', 62: 'SN1bc', 95: 'SLSN1', 15: 'TDE', 64: 'KN', 88: 'AGN', 92: 'RRL', 65: 'M-dwarf', 16: 'EB', 53: 'Mira', 6: 'Microlens'}

In [5]:
y_train = []
for i,obj_class in enumerate(x_labels):
    #class_ = meta_[meta_['object_id'] == obj_id]['target'].values[0]
    
    if category_mapping[obj_class] == 'AGN':
        y_train.append(1)
    else:
        y_train.append(0)
    
y_train = np.array(y_train)

In [8]:
# Model / data parameters
num_classes = 2
input_shape = (256, 1300, 6)

y_train = keras.utils.to_categorical(y_train, num_classes)

In [11]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 1298, 32)     1760      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 649, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 647, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 323, 64)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1281664)           0         
                                                                 
 dropout (Dropout)           (None, 1281664)           0

2022-06-23 12:01:21.838698: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
batch_size = 32
epochs = 3

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/3
4/4 [==============================] - 24s 5s/step - loss: 62.6176 - accuracy: 0.8261 - val_loss: 82.0613 - val_accuracy: 0.8462
Epoch 2/3
4/4 [==============================] - 21s 5s/step - loss: 36.2692 - accuracy: 0.9130 - val_loss: 14.7818 - val_accuracy: 0.9231
Epoch 3/3
4/4 [==============================] - 20s 5s/step - loss: 10.5563 - accuracy: 0.8696 - val_loss: 3.4845e-06 - val_accuracy: 1.0000
